In [1]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [2]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.utcnow())
print(f"{current_date}")

2025-05-11 09:45:13.296661


In [3]:
type(current_date)

pandas._libs.tslibs.timestamps.Timestamp

In [4]:
from datetime import datetime, timedelta, timezone
current_date = pd.to_datetime(datetime.now(timezone.utc)).floor("4h")

In [5]:
current_date.to_datetime64()

numpy.datetime64('2025-05-11T08:00:00.000000000')

In [6]:
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=29)

In [7]:
fetch_data_to

Timestamp('2025-05-11 08:00:00+0000', tz='UTC')

In [8]:
fetch_data_from

Timestamp('2025-04-12 08:00:00+0000', tz='UTC')

In [15]:
from datetime import datetime, timedelta
from typing import Union
import pandas as pd
from src.data_utils import load_and_process_taxi_data

def fetch_batch_raw_data(from_date: Union[datetime, str], to_date: Union[datetime, str]) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e., 1 year).

    Args:
        from_date (datetime or str): The start date for the data batch.
        to_date (datetime or str): The end date for the data batch.

    Returns:
        pd.DataFrame: A DataFrame containing the simulated production data.
    """
    # Convert string inputs to datetime if necessary
    if isinstance(from_date, str):
        from_date = datetime.fromisoformat(from_date)
    if isinstance(to_date, str):
        to_date = datetime.fromisoformat(to_date)

    # Validate input dates
    if not isinstance(from_date, datetime) or not isinstance(to_date, datetime):
        raise ValueError("Both 'from_date' and 'to_date' must be datetime objects or valid ISO format strings.")
    if from_date >= to_date:
        raise ValueError("'from_date' must be earlier than 'to_date'.")

    # Shift dates back by 52 weeks (1 year)
    historical_from_date = from_date - timedelta(weeks=16)
    historical_to_date = to_date - timedelta(weeks=16)

    # Load and filter data for the historical period
    rides_from = load_and_process_taxi_data(year=historical_from_date.year, months=[historical_from_date.month])
    rides_from = rides_from[rides_from.started_at >= historical_from_date]

    if historical_to_date.month != historical_from_date.month:
        rides_to = load_and_process_taxi_data(year=historical_to_date.year, months=[historical_to_date.month])
        rides_to = rides_to[rides_to.started_at < historical_to_date]
        # Combine the filtered data
        rides = pd.concat([rides_from, rides_to], ignore_index=True)
    else:
        rides = rides_from
    # Shift the data forward by 52 weeks to simulate recent data
    rides['started_at'] += timedelta(weeks=16)

    # Sort the data for consistency
    rides.sort_values(by=['pickup_location_id', 'started_at'], inplace=True)

    return rides

In [16]:
fetch_data_from = pd.to_datetime(fetch_data_from).tz_localize(None)
fetch_data_to = pd.to_datetime(fetch_data_to).tz_localize(None)
rides = fetch_batch_raw_data(fetch_data_from, fetch_data_to)

File already exists for 2024-12.
Loading data for 2024-12...
Total records: 2,302,229
Valid records: 74,450
Records dropped: 2,227,779 (96.77%)
Successfully processed data for 2024-12.
Combining all monthly data...
Data loading and processing complete!
File already exists for 2025-01.
Loading data for 2025-01...
Total records: 2,119,110
Valid records: 69,006
Records dropped: 2,050,104 (96.74%)
Successfully processed data for 2025-01.
Combining all monthly data...
Data loading and processing complete!


In [17]:
rides

,started_at,pickup_location_id
1948,2025-04-12 08:25:21.169,5626
2060,2025-04-12 08:26:33.290,5626
15214,2025-04-12 08:27:12.861,5626
1207,2025-04-12 08:27:12.927,5626
10423,2025-04-12 08:31:24.653,5626
...,...,...
45292,2025-05-11 07:24:51.667,6072
53129,2025-05-11 07:43:52.635,6072
46320,2025-05-11 07:44:09.333,6072
52634,2025-05-11 07:53:33.477,6072


In [18]:
from src.data_utils import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

In [19]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2025-04-12 08:00:00,5626,62
1,2025-04-12 12:00:00,5626,104
2,2025-04-12 16:00:00,5626,86
3,2025-04-12 20:00:00,5626,31
4,2025-04-13 00:00:00,5626,19
...,...,...,...
865,2025-05-10 12:00:00,6072,85
866,2025-05-10 16:00:00,6072,59
867,2025-05-10 20:00:00,6072,54
868,2025-05-11 00:00:00,6072,23


In [20]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 870 entries, 0 to 869
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   pickup_hour         870 non-null    datetime64[ns]
 1   pickup_location_id  870 non-null    int16         
 2   rides               870 non-null    int16         
dtypes: datetime64[ns](1), int16(2)
memory usage: 10.3 KB


In [ ]:
#

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174696 entries, 0 to 174695
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         174696 non-null  datetime64[ns]
 1   pickup_location_id  174696 non-null  int64         
 2   rides               174696 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 4.0 MB


In [21]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)



2025-05-11 06:03:20,739 INFO: Initializing external client
2025-05-11 06:03:20,739 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-05-11 06:03:21,385 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214695


In [22]:
feature_group.schema

[Feature('pickup_hour', 'timestamp', None, True, False, None, None, 1458598),
 Feature('pickup_location_id', 'int', None, True, False, None, None, 1458598),
 Feature('rides', 'int', None, False, False, None, None, 1458598)]

In [23]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |██████████| Rows 870/870 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: citibike_hourly_features_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1214695/jobs/named/citibike_hourly_features_1_offline_fg_materialization/executions


(Job('citibike_hourly_features_1_offline_fg_materialization', 'SPARK'), None)